In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
20,model_15_9_5,0.990963,0.994220,0.938871,0.983715,0.010002,0.009437,0.030089,0.019155,"Hidden Size=[40], regularizer=0.01, learning_r..."
23,model_15_9_4,0.990613,0.995470,0.942655,0.985379,0.010389,0.007396,0.028226,0.017198,"Hidden Size=[40], regularizer=0.01, learning_r..."
28,model_15_9_3,0.990027,0.996601,0.946431,0.986953,0.011037,0.005550,0.026368,0.015347,"Hidden Size=[40], regularizer=0.01, learning_r..."
35,model_15_9_2,0.989130,0.997529,0.950105,0.988359,0.012030,0.004034,0.024559,0.013693,"Hidden Size=[40], regularizer=0.01, learning_r..."
40,model_5_9_8,0.988881,0.979804,0.924282,0.976878,0.012305,0.031359,0.016306,0.024275,"Hidden Size=[15], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
287,model_15_5_2,0.980791,0.988145,0.973077,0.984175,0.021258,0.014965,0.015500,0.015216,"Hidden Size=[40], regularizer=0.01, learning_r..."
289,model_10_9_4,0.980737,0.950440,0.992632,0.982566,0.021318,0.036422,0.012213,0.025030,"Hidden Size=[25], regularizer=0.1, learning_ra..."
297,model_15_5_4,0.980515,0.972435,0.938371,0.963479,0.021565,0.034793,0.035479,0.035116,"Hidden Size=[40], regularizer=0.01, learning_r..."
300,model_8_8_1,0.980439,0.964710,0.971882,0.974264,0.021648,0.002825,0.035260,0.018089,"Hidden Size=[20], regularizer=0.1, learning_ra..."
